
# 🧠 Notebook: Generación del Dataset Consolidado `df_final.csv`

## 📌 Objetivo General
Integrar múltiples archivos `.csv` ubicados en la carpeta `data/`, correspondientes a diferentes deportes, en un único archivo llamado `df_final.csv`, que será la base de análisis para la monitorización de los datos deportivos.


## 🥇 Paso 1: Cargar todos los archivos CSV

In [47]:
from pathlib import Path
import pandas as pd

# ✅ Usamos la carpeta actual (donde está el notebook) como base
data_folder = Path(".")  # Este notebook está en data/, así que buscamos en la misma carpeta
output_folder = data_folder / "concat"
output_folder.mkdir(parents=True, exist_ok=True)  # Crea 'data/concat' si no existe

# 🔍 Buscar todos los archivos CSV en esta carpeta
csv_files = list(data_folder.glob("*.csv"))

# 📋 Mostrar archivos encontrados para verificación
print("📁 Archivos encontrados:", [f.name for f in csv_files])


📁 Archivos encontrados: ['Atletismo_2024.csv', 'Badminton_2024.csv', 'Baloncesto_Femenino_2024.csv', 'Baloncesto_Masculino_2024.csv', 'Balonmano_Femenino_2024.csv', 'Balonmano_Masculino_2024.csv', 'Gimnasia_2024.csv']


## 🧩 Paso 2: Lectura individual e identificación del deporte

In [48]:

dataframes = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        deporte = file.stem.split("_")[0]
        df.insert(0, "Deporte", deporte)  # Agregar columna con el nombre del deporte
        dataframes.append(df)
    except Exception as e:
        print(f"❌ Error leyendo {file.name}: {e}")


In [49]:
dataframes = []

for archivo in csv_files:
    nombre_archivo = archivo.stem  # ej: Atletismo_2024

    # === Extraer metadatos desde el nombre del archivo ===
    deporte = nombre_archivo.split("_")[0].capitalize()

    genero_archivo = None
    if "Femenino" in nombre_archivo:
        genero_archivo = "Femenino"
    elif "Masculino" in nombre_archivo:
        genero_archivo = "Masculino"

    try:
        # === Leer archivo ===
        df = pd.read_csv(archivo)
        df.columns = df.columns.str.strip().str.upper()

        # === Añadir o completar columna GENERO solo si aplica ===
        if "GENERO" in df.columns:
            df["GENERO"] = df["GENERO"].fillna("").astype(str).str.strip().str.capitalize()
            if genero_archivo and df["GENERO"].eq("").all():
                df["GENERO"] = genero_archivo
        elif genero_archivo:
            df["GENERO"] = genero_archivo
        # Si no tiene género detectado y no estaba en el archivo, se deja sin columna

        # === Agregar columna DEPORTE ===
        df["DEPORTE"] = deporte

        dataframes.append(df)

        print(f"✅ Procesado: {archivo.name}")

    except Exception as e:
        print(f"❌ Error con {archivo.name}: {e}")


✅ Procesado: Atletismo_2024.csv
✅ Procesado: Badminton_2024.csv
✅ Procesado: Baloncesto_Femenino_2024.csv
✅ Procesado: Baloncesto_Masculino_2024.csv
✅ Procesado: Balonmano_Femenino_2024.csv
✅ Procesado: Balonmano_Masculino_2024.csv
✅ Procesado: Gimnasia_2024.csv


In [50]:
df_final = pd.concat(dataframes, ignore_index=True)
df_final.to_csv(output_folder / "df_final.csv", index=False)


## 🔗 Paso 3: Concatenación flexible (merge por outer join)

In [51]:

if dataframes:
    df_final = pd.concat(dataframes, ignore_index=True, sort=False)
    print(f"✅ DataFrame concatenado con {df_final.shape[0]} filas y {df_final.shape[1]} columnas.")
else:
    print("⚠️ No hay archivos válidos para concatenar.")


✅ DataFrame concatenado con 31979 filas y 36 columnas.


## 📦 Paso 4: Concatenar y exportar los datos finales
En este paso, unificamos todos los DataFrames que se cargaron correctamente en pasos anteriores. Si ninguno de los archivos fue válido, detenemos el proceso con un mensaje claro y detallado. De lo contrario, exportamos el dataset final a `data/concat/df_final.csv` para su uso posterior.

In [52]:

if dataframes:
    output_path = output_folder / "df_final_2.csv"
    df_final.to_csv(output_path, index=False)
    print(f"📁 Archivo guardado en: {output_path}")


PermissionError: [Errno 13] Permission denied: 'concat\\df_final_2.csv'

## ✅ Resultado

El DataFrame final contiene todas las columnas y filas sin perder información, con nombre del deporte en la primera columna y estructura homogénea para su análisis posterior.